# AER - Access Entitlement Review Reporter (Optimized v3)

**Updates:**
- **Tree Selector**: Pick specific Apps under Categories (Q3, Q4).
- **Integrated Reporting**: Reports generated immediately after processing in `output/YYYY-MM-DD/`.
- **Detailed Logging**: Shows user count per App & success logs per user.
- **UI Improvements**: Better readability for Email rows.

In [ ]:
# === Cell 1: 設定與認證 (互動式視窗登入 & UTF-8 Logging) ===
import os
import sys
import io
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication
import requests

# 讀取 .env
load_dotenv()

# === Logging 設定 ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear()
logger.setLevel(logging.INFO)

def _get_console_stream():
    s = getattr(sys, "stdout", None)
    try:
        if s and hasattr(s, "reconfigure"):
            s.reconfigure(encoding="utf-8")
            if hasattr(sys.stderr, "reconfigure"):
                sys.stderr.reconfigure(encoding="utf-8")
            return s
    except Exception: pass
    try:
        buf = getattr(s, "buffer", None)
        if buf: return io.TextIOWrapper(buf, encoding="utf-8", errors="replace")
    except Exception: pass
    return None

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
ch = logging.StreamHandler(_get_console_stream())
ch.setFormatter(formatter)
logger.addHandler(ch)
fh = logging.FileHandler(log_file, encoding="utf-8")
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review"
BASE_PATH = APP_NAME
SENDER_EMAIL = os.getenv("SENDER_EMAIL")

SCOPES = [
    "User.Read.All",         # 讀取你的個人檔案
    "Mail.Send",
    "Mail.Read",         # 讀取你的 Email (Copilot 核心功能)
    "Files.Read.All",    # 讀取 OneDrive/SharePoint 檔案
    "Sites.Read.All",    # 搜尋 SharePoint 網站
    "People.Read.All", # 備用
    "OnlineMeetingTranscript.Read.All",
    "Chat.Read",
    "ChannelMessage.Read.All",
    "ExternalItem.Read.All"
]

app = PublicClientApplication(CLIENT_ID, authority=f"https://login.microsoftonline.com/{TENANT_ID}")

print("🚀 正在開啟瀏覽器進行登入...")
interactive_result = app.acquire_token_interactive(scopes=SCOPES, prompt="select_account")

if "access_token" not in interactive_result:
    raise RuntimeError(f"登入失敗: {interactive_result.get('error_description')}")

headers = {"Authorization": f"Bearer {interactive_result['access_token']}"}
logger.info("✅ 登入成功")
logger.info(f"Log 檔案: {log_file}")

In [ ]:
# === Cell 2: SharePoint API & App Tree Selector (Cache Toggle Added) ===
import ipywidgets as widgets
from IPython.display import display, clear_output

# ... (SharePoint API Functions: get_site_id, list_folders, list_excel_files, download_file, get_file_audit_info 維持不變) ...
# 為節省篇幅，請保留原有的 API 函式定義，僅修改下方的 UI 建構部分

# --- 3. Tree Selector UI ---
TARGET_APPS = [] 
USE_CACHE = True # Default

def create_app_selector():
    print(f"📂 正在讀取根目錄: {BASE_PATH} ...")
    try:
        categories = list_folders(site_id, BASE_PATH)
    except Exception as e:
        print(f"❌ 讀取失敗: {e}"); return

    ui_container = widgets.VBox()
    app_checkboxes = []
    
    # ★ Improvement #5: 新增 Cache Toggle
    chk_cache = widgets.Checkbox(value=True, description="⚡ 使用快取 (Use Cache)", indent=False)
    ui_container.children += (widgets.HTML("<h3>📂 App 選擇器</h3>"), chk_cache, widgets.HTML("<hr>"))

    for cat in categories:
        if cat['name'] in ["Forms", "_private"] or "audit" in cat['name'].lower(): continue
        cat_label = widgets.HTML(f"<b>📁 {cat['name']}</b>", layout=widgets.Layout(width='100px'))
        btn_expand = widgets.Button(description="➕ 展開", button_style='info', layout=widgets.Layout(width='80px'))
        app_list_box = widgets.VBox(layout=widgets.Layout(margin='0 0 0 30px', display='none'))
        
        def on_expand(b, cat_name=cat['name'], container=app_list_box, btn=btn_expand):
            if btn.description == "➕ 展開":
                btn.description = "⏳"
                sub_path = f"{BASE_PATH}/{cat_name}"
                try:
                    apps = list_folders(site_id, sub_path)
                    checks = []
                    if not apps: checks.append(widgets.Label("(無資料夾)"))
                    for app in apps:
                        if app['name'] in ["Forms", "_private"]: continue
                        cb = widgets.Checkbox(value=False, description=app['name'], indent=False)
                        cb.app_data = (cat_name, app['name'], f"{sub_path}/{app['name']}")
                        checks.append(cb); app_checkboxes.append(cb)
                    container.children = tuple(checks); container.layout.display = 'block'; btn.description = "➖ 收起"
                except Exception as e:
                    container.children = (widgets.Label(f"Error: {e}"),); btn.description = "❌"
            else:
                if container.layout.display == 'none': container.layout.display = 'block'; btn.description = "➖ 收起"
                else: container.layout.display = 'none'; btn.description = "➕ 展開"

        btn_expand.on_click(on_expand)
        ui_container.children += (widgets.HBox([btn_expand, cat_label]), app_list_box)

    btn_confirm = widgets.Button(description="✅ 確認選擇", button_style='success', layout=widgets.Layout(margin='20px 0'))
    output_area = widgets.Output()

    def on_confirm(b):
        global TARGET_APPS, USE_CACHE
        TARGET_APPS = [cb.app_data for cb in app_checkboxes if cb.value]
        USE_CACHE = chk_cache.value # Update Global Variable
        
        with output_area:
            clear_output()
            if not TARGET_APPS: print("⚠️ 未選擇任何 App！")
            else:
                cache_status = "開啟" if USE_CACHE else "關閉 (強制重新讀取)"
                print(f"🎯 已選擇 {len(TARGET_APPS)} 個 Apps | 快取: {cache_status}")
                print("⏳ 請執行 Cell 4 開始處理。")

    btn_confirm.on_click(on_confirm)
    display(ui_container, btn_confirm, output_area)

create_app_selector()

In [ ]:
# === Cell 3: Excel 讀取函數 (保持不變) ===
from openpyxl import load_workbook
from io import BytesIO

COL_REVIEWER = "Reviewer"
COL_RESPONSE = "Reviewer's Response"
COL_DETAILS = "Details of Access change"

def read_visible_rows(excel_bytes: bytes, reviewer_name: str, file_name: str, folder_url: str) -> list[dict]:
    wb = load_workbook(BytesIO(excel_bytes))
    ws = wb.active
    
    header_row = [cell.value for cell in ws[1]]
    col_map = {str(name).strip(): idx for idx, name in enumerate(header_row) if name}
    
    reviewer_col = col_map.get(COL_REVIEWER)
    response_col = col_map.get(COL_RESPONSE)
    details_col = col_map.get(COL_DETAILS)
    
    if reviewer_col is None or response_col is None:
        raise ValueError(f"Missing columns. Found: {list(col_map.keys())}")
    
    has_filter = any(ws.row_dimensions[i].hidden for i in range(2, ws.max_row + 1) if i in ws.row_dimensions)
    results = []
    
    for row_idx in range(2, ws.max_row + 1):
        if has_filter and ws.row_dimensions.get(row_idx) and ws.row_dimensions[row_idx].hidden:
            continue
        row = [cell.value for cell in ws[row_idx]]
        
        reviewer = row[reviewer_col] if reviewer_col < len(row) else None
        response = row[response_col] if response_col < len(row) else None
        details = row[details_col] if details_col is not None and details_col < len(row) else None
        
        if not has_filter and (reviewer is None or str(reviewer).strip().lower() != reviewer_name.lower()):
            continue
        
        results.append({
            "reviewer": reviewer_name, "response": response, "details": details,
            "is_missing": (response is None or str(response).strip() == ""),
            "row_number": row_idx, "file_name": file_name, "folder_url": folder_url
        })
    return results

In [ ]:
# === Cell 4: 整合版 (Fix Stats Population & Enhanced Keywords) ===
import pandas as pd
import re, time, json, os, requests
import ipywidgets as widgets
from datetime import datetime
from openpyxl.utils import get_column_letter
from IPython.display import display, HTML, clear_output

# ==========================================
# PART 1: CONFIG & PERSISTENCE
# ==========================================
CACHE_FILE = "aer_cache.json"
NOTES_FILE = "aer_manual_notes.json"
cache_updated = False
EXCLUDED_FOLDERS = ["forms", "_private", "user listings", "audit logs", "audit"]

def load_json(file_path):
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f: return json.load(f)
        except: return {}
    return {}

def save_json(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f: json.dump(data, f, indent=2, ensure_ascii=False)

local_cache = load_json(CACHE_FILE)
manual_data_store = load_json(NOTES_FILE)

# ★ Improvement #6: 強化的關鍵字判斷邏輯
def get_row_stats(txt):
    txt = str(txt).lower().strip() if txt else ""
    
    # 定義關鍵字群組
    kw_approve = ['approv', 'retain', 'keep', 'confirm', 'verif', 'yes', 'ok', 'maintain', 'continue', 'active', 'valid']
    kw_deny    = ['denied', 'deny', 'remove', 'delete', 'revok', 'disabl', 'terminat', 'reject', 'cancel', 'no', 'inactiv']
    kw_change  = ['change', 'modif', 'updat', 'correct', 'edit', 'adjust', 'alter']
    
    return {
        "is_appr": 1 if any(k in txt for k in kw_approve) else 0,
        "is_deny": 1 if any(k in txt for k in kw_deny) else 0,
        "is_chg":  1 if any(k in txt for k in kw_change) else 0
    }

# ==========================================
# PART 2: SCANNING ENGINE
# ==========================================
if 'TARGET_APPS' not in globals() or not TARGET_APPS:
    logger.warning("⚠️ 請先在 Cell 2 選擇 App！")
    TARGET_APPS = []

all_responses = []
errors = []

for category, current_app_name, current_path in TARGET_APPS:
    try:
        raw_folders = list_folders(site_id, current_path)
        reviewers = []
        for r in raw_folders:
            r_name_lower = r['name'].lower()
            if r_name_lower not in EXCLUDED_FOLDERS and "audit" not in r_name_lower and "listing" not in r_name_lower:
                reviewers.append(r)
        
        logger.info(f"🚀 Processing: {current_app_name} | Users: {len(reviewers)}")
        
        for folder in reviewers:
            reviewer_name = folder["name"]
            folder_url = folder["webUrl"]
            folder_path = f"{current_path}/{reviewer_name}"
            
            for attempt in range(2):
                try:
                    excel_files = list_excel_files(site_id, folder_path)
                    target_files = []
                    search_prefix = f"{current_app_name} User".lower()
                    search_user = reviewer_name.lower()
                    
                    for f in excel_files:
                        f_name = f["name"].lower()
                        if f_name.startswith(search_prefix) and search_user in f_name:
                            target_files.append(f)
                    
                    if not target_files:
                        raise FileNotFoundError(f"No file matching format: '{current_app_name} User * {reviewer_name}.xlsx'")
                    
                    target_file = target_files[0]
                    file_name = target_file["name"]
                    remote_mod_time = target_file.get("lastModifiedDateTime")
                    meta_created_time = target_file.get("createdDateTime")
                    
                    # --- CACHE CHECK ---
                    is_hit = False
                    cache_key = f"{category}|{current_app_name}|{reviewer_name}"
                    
                    if USE_CACHE:
                        cached_entry = local_cache.get(cache_key)
                        if isinstance(cached_entry, dict) and cached_entry.get('last_mod') == remote_mod_time:
                            is_hit = True
                            stats_snapshot = cached_entry.get('stats', {'Appr':0, 'Deny':0, 'Chg':0})
                            audit_snapshot = cached_entry.get('audit', {'log':'Cached', 'creator':'Cached', 'modifier':'Cached', 'created_ts': None})
                        
                    if is_hit:
                        logger.info(f"  ⏭️ Skipped (Cached): {reviewer_name}")
                        final_created_date = audit_snapshot.get('created_ts') or meta_created_time
                        
                        all_responses.append({
                            "Category": category, "App_Name": current_app_name,
                            "reviewer": reviewer_name, "response": "Cached - Completed", 
                            "details": "Cached", 
                            "is_missing": False, "row_number": 0, "file_name": file_name, "folder_url": folder_url,
                            "Last_Modified": remote_mod_time,
                            "File_Created_Date": final_created_date,
                            "Audit_Log": audit_snapshot.get('log', ''), 
                            "File_Creator": audit_snapshot.get('creator', ''), 
                            "File_Modifier": audit_snapshot.get('modifier', ''),
                            "stats_appr": stats_snapshot['Appr'],
                            "stats_deny": stats_snapshot['Deny'],
                            "stats_chg": stats_snapshot['Chg']
                        })
                        break 
                    
                    # --- DOWNLOAD & PARSE ---
                    content = download_file(site_id, f"{folder_path}/{file_name}")
                    rows = read_visible_rows(content, reviewer_name, file_name, folder_url)
                    audit_info = get_file_audit_info(site_id, target_file["id"])
                    final_created_date = audit_info.get('created_ts') or meta_created_time
                    
                    miss_cnt = 0
                    s_appr, s_deny, s_chg = 0, 0, 0
                    
                    for r in rows:
                        if r['is_missing']: miss_cnt += 1
                        # ★ Improvement #6: 使用更新後的 stats 邏輯
                        st = get_row_stats(r['response'])
                        s_appr += st['is_appr']; s_deny += st['is_deny']; s_chg += st['is_chg']
                        
                        r.update({
                            "Category": category, "App_Name": current_app_name,
                            "Last_Modified": remote_mod_time,
                            "File_Created_Date": final_created_date,
                            "Audit_Log": audit_info['log'],
                            "File_Creator": audit_info['creator'],
                            "File_Modifier": audit_info['modifier'],
                            "stats_appr": st['is_appr'], "stats_deny": st['is_deny'], "stats_chg": st['is_chg']
                        })
                    
                    all_responses.extend(rows)
                    logger.info(f"  ✅ Read: {reviewer_name} (Miss: {miss_cnt}, A:{s_appr}, D:{s_deny}, C:{s_chg})")
                    
                    if miss_cnt == 0 and len(rows) > 0:
                        local_cache[cache_key] = {
                            "last_mod": remote_mod_time,
                            "stats": {"Appr": s_appr, "Deny": s_deny, "Chg": s_chg},
                            "audit": audit_info
                        }
                        cache_updated = True
                    break 

                except Exception as e:
                    if attempt == 0: time.sleep(1)
                    else:
                        err_msg = str(e) if "No file matching" in str(e) else f"Error: {e}"
                        logger.error(f"  ❌ Failed {reviewer_name}: {err_msg}")
                        errors.append({
                            "Category": category, "App_Name": current_app_name, 
                            "reviewer": reviewer_name, "error": err_msg, "folder_url": folder_url
                        })
    except Exception as e:
        logger.error(f"❌ App Error ({current_app_name}): {e}")

if cache_updated:
    save_json(CACHE_FILE, local_cache)
    logger.info("💾 Cache updated safely.")

# ==========================================
# PART 3: AGGREGATION & DASHBOARD
# ==========================================
df = pd.DataFrame(all_responses)
today_str = datetime.now().strftime("%Y-%m-%d")
output_dir = f"output/{today_str}"
os.makedirs(output_dir, exist_ok=True)

unified_data = {}
def get_app_node(cat, app):
    key = f"{cat} > {app}"
    if key not in unified_data:
        saved_app = manual_data_store.get(key, {})
        unified_data[key] = {
            "Category": cat, "App_Name": app, 
            "status_manual": saved_app.get("app_status", "Calculated"), 
            "note_manual": saved_app.get("app_note", ""), 
            "reviewers": {}, "stats": {"total_users": 0, "completed_users": 0}
        }
    return unified_data[key]

# --- Aggregation Logic ---
if not df.empty:
    stats = df.groupby(["Category", "App_Name", "reviewer"]).agg(
        missing=("is_missing", "sum"),
        approved=("stats_appr", "sum"),
        denied=("stats_deny", "sum"),
        changed=("stats_chg", "sum"),
        f_creator=("File_Creator", "first"),
        f_modifier=("File_Modifier", "first"),
        audit=("Audit_Log", "first")
    ).reset_index()

    for _, row in stats.iterrows():
        node = get_app_node(row['Category'], row['App_Name'])
        rev_key = row['reviewer']
        is_done = (row['missing'] == 0)
        node['stats']['total_users'] += 1
        if is_done: node['stats']['completed_users'] += 1
        
        saved_rev = manual_data_store.get(f"{node['Category']} > {node['App_Name']}", {}).get("reviewers", {}).get(rev_key, {})
        status_calc = "✅ Completed" if is_done else f"❌ Pending: {row['missing']}"
        
        # ★ Improvement #6: 這裡顯示的 detail_str 現在會基於更準確的統計
        detail_str = f"Appr:{int(row['approved'])} | Deny:{int(row['denied'])} | Chg:{int(row['changed'])}"
        
        node['reviewers'][rev_key] = {
            "status_calc": status_calc, "detail_str": detail_str,
            "missing_cnt": row['missing'],
            "stats": {"Appr": row['approved'], "Deny": row['denied'], "Chg": row['changed']},
            "audit_info": {"creator": row['f_creator'], "modifier": row['f_modifier'], "log": row['audit']},
            "override": saved_rev.get("override", "-"), "note": saved_rev.get("note", ""),
            "error_msg": "", "folder_url": df[(df['App_Name'] == row['App_Name']) & (df['reviewer'] == rev_key)].iloc[0].get('folder_url', '#')
        }

# Process Errors for Dashboard
for err in errors:
    node = get_app_node(err['Category'], err['App_Name'])
    rev_key = err['reviewer']
    node['stats']['total_users'] += 1
    saved_rev = manual_data_store.get(f"{node['Category']} > {node['App_Name']}", {}).get("reviewers", {}).get(rev_key, {})
    if rev_key in node['reviewers']:
        node['reviewers'][rev_key]['error_msg'] += f"; {err['error']}"
        node['reviewers'][rev_key]['status_calc'] = "⚠️ Partial Error"
    else:
        node['reviewers'][rev_key] = {
            "status_calc": "⛔ Failed to Read",
            "detail_str": f"<span style='color:red'>{err['error']}</span>",
            "missing_cnt": -1, "stats": {"Appr": 0, "Deny": 0, "Chg": 0},
            "audit_info": {"creator": "Error", "modifier": "Error", "log": ""},
            "override": saved_rev.get("override", "-"), "note": saved_rev.get("note", ""),
            "error_msg": err['error'], "folder_url": err.get('folder_url', '#')
        }

# ==========================================
# PART 4: DASHBOARD UI & EXPORT LOGIC
# ==========================================
widget_store = {} 

def build_dashboard():
    container = widgets.VBox(layout=widgets.Layout(width='100%'))
    btn_export = widgets.Button(description="💾 Save App Reports", button_style='success', icon='file-excel', layout=widgets.Layout(width='250px'))
    lbl_out = widgets.Label(value="")
    
    app_widgets = []
    
    for app_key in sorted(unified_data.keys()):
        app_data = unified_data[app_key]
        comp = app_data['stats']['completed_users']
        tot = app_data['stats']['total_users']
        pct = int((comp / tot * 100)) if tot > 0 else 0
        header_color = "#e6ffe6" if comp == tot and tot > 0 else "#fff3e6"
        
        w_lbl = widgets.HTML(f"<b>📂 {app_key}</b> &nbsp;&nbsp; <span style='background:white; border:1px solid #ccc; padding:2px 6px; border-radius:4px; font-size:11px'>✅ {comp}/{tot} ({pct}%)</span>", layout=widgets.Layout(width='350px'))
        w_status = widgets.Dropdown(options=["Calculated", "Force Completed", "N/A", "Action Required"], value=app_data['status_manual'], layout=widgets.Layout(width='150px'))
        w_note = widgets.Text(value=app_data['note_manual'], placeholder="App Note...", layout=widgets.Layout(width='200px'))
        
        widget_store[app_key] = {"data": app_data, "w_stat": w_status, "w_note": w_note, "revs": {}}
        
        rev_rows = [widgets.HBox([
            widgets.HTML("<b>Reviewer</b>", layout=widgets.Layout(width='180px')),
            widgets.HTML("<b>Status</b>", layout=widgets.Layout(width='140px')),
            widgets.HTML("<b>Breakdown / Error</b>", layout=widgets.Layout(width='250px')),
            widgets.HTML("<b>Override</b>", layout=widgets.Layout(width='120px')),
            widgets.HTML("<b>Note</b>", layout=widgets.Layout(width='180px'))
        ])]
        
        for r_name, r_d in app_data['reviewers'].items():
            color = "#333"; bg = "white"
            if "Failed" in r_d['status_calc']: color = "red"; bg="#fff0f0"
            elif "Pending" in r_d['status_calc']: color = "darkorange"
            elif "Completed" in r_d['status_calc']: color = "green"
            
            w_r_name = widgets.HTML(f"<a href='{r_d['folder_url']}' target='_blank' style='color:{color}; text-decoration:none'>{r_name}</a>", layout=widgets.Layout(width='180px'))
            w_r_st = widgets.HTML(f"<span style='color:{color}'>{r_d['status_calc']}</span>", layout=widgets.Layout(width='140px'))
            w_r_det = widgets.HTML(f"<span style='font-size:11px'>{r_d['detail_str']}</span>", layout=widgets.Layout(width='250px'))
            w_ovr = widgets.Dropdown(options=["-", "Mark Done", "Extension", "Escalated"], value=r_d['override'], layout=widgets.Layout(width='120px'))
            w_rn = widgets.Text(value=r_d['note'], placeholder="...", layout=widgets.Layout(width='180px'))
            
            widget_store[app_key]['revs'][r_name] = {"w_ovr": w_ovr, "w_note": w_rn, "sys": r_d}
            rev_rows.append(widgets.HBox([w_r_name, w_r_st, w_r_det, w_ovr, w_rn], layout=widgets.Layout(background_color=bg, border='1px solid #f4f4f4')))
            
        app_head = widgets.HBox([w_lbl, w_status, w_note], layout=widgets.Layout(background_color=header_color, padding="8px", border="1px solid #ccc"))
        app_body = widgets.VBox(rev_rows, layout=widgets.Layout(padding="5px 0 10px 20px"))
        app_widgets.append(widgets.VBox([app_head, app_body], layout=widgets.Layout(margin="10px 0")))

    def export(b):
        b.disabled=True; b.description="Saving..."
        ts = datetime.now().strftime("%H%M%S")
        persist_store = load_json(NOTES_FILE)
        saved_files = []
        
        for k, v in widget_store.items():
            app_s = v['w_stat'].value; app_n = v['w_note'].value
            persist_store[k] = {"app_status": app_s, "app_note": app_n, "reviewers": {}}
            for rk, rv in v['revs'].items():
                persist_store[k]["reviewers"][rk] = {"override": rv['w_ovr'].value, "note": rv['w_note'].value}
        save_json(NOTES_FILE, persist_store)
        
        for app_key, widget_data in widget_store.items():
            app_name = widget_data['data']['App_Name']
            category = widget_data['data']['Category']
            app_errors = [e for e in errors if e['App_Name'] == app_name]
            suffix = "_Incomplete" if app_errors else ""
            
            if df.empty: app_rows = []
            else: app_rows = df[(df['Category'] == category) & (df['App_Name'] == app_name)].to_dict('records')
            
            if not app_rows and app_errors:
                 app_rows = [{"reviewer": e['reviewer'], "details": f"Error: {e['error']}", "response": "Error"} for e in app_errors]

            final_data = []
            for row in app_rows:
                rev_name = row.get('reviewer')
                dash_rev = widget_data['revs'].get(rev_name, {})
                sys_status = dash_rev.get('sys', {}).get('status_calc', 'Unknown')
                ovr = dash_rev.get('w_ovr', widgets.Dropdown()).value if dash_rev else "-"
                final_status = widget_data['w_stat'].value if widget_data['w_stat'].value != "Calculated" else (ovr if ovr != "-" else sys_status)
                
                final_data.append({
                    "Reviewer": rev_name,
                    "File Name": row.get('file_name', ''),
                    "Row Num": row.get('row_number', ''),
                    "Reviewer Response": row.get('response', ''), 
                    "Details of Access Change": row.get('details', ''), 
                    "Final Status": final_status,
                    "Manual Override": ovr,
                    "Admin Note": dash_rev.get('w_note', widgets.Text()).value if dash_rev else "",
                    "File Created": row.get('File_Created_Date', ''),
                    "Last Modified": row.get('Last_Modified', ''),
                    "Full Audit Log": row.get('Audit_Log', '')
                })
            
            if final_data:
                safe_name = re.sub(r'[\\/*?:"<>|]', "", app_name)
                f_name = f"{output_dir}/Report_{safe_name}{suffix}.xlsx"
                pd.DataFrame(final_data).to_excel(f_name, index=False)
                saved_files.append(f"<a href='{f_name}' target='_blank'>{safe_name}{suffix}</a>")
        
        lbl_out.value = f"Saved {len(saved_files)} files."
        display(HTML("✅ Saved: " + ", ".join(saved_files)))
        b.disabled=False; b.description="💾 Save App Reports"

    btn_export.on_click(export)
    display(widgets.HBox([btn_export, lbl_out]))
    container.children = tuple(app_widgets)
    display(container)

if unified_data:
    build_dashboard()
else:
    display(HTML("<b>⚠️ No data found.</b>"))

In [ ]:
# === Cell 6: Email Notification (Creation Date Logic) ===
import ipywidgets as widgets
from urllib.parse import quote
from datetime import datetime, timedelta
import requests
import pandas as pd
from IPython.display import display, clear_output

# --- 1. Helper Functions ---
def get_email(name): 
    if not name: return ""
    try:
        clean = quote(name.split("(")[0].strip())
        url = f"https://graph.microsoft.com/v1.0/users?$filter=startswith(displayName,'{clean}')&$select=mail,userPrincipalName"
        res = requests.get(url, headers=headers).json().get('value')
        if res: return res[0].get('mail') or res[0].get('userPrincipalName')
    except: pass
    return ""

def calculate_due_date(iso_date_str):
    if not iso_date_str or pd.isna(iso_date_str): return "ASAP"
    try:
        dt_str = str(iso_date_str).replace('Z', '').split('.')[0]
        dt = datetime.fromisoformat(dt_str)
        # ★ 規則：從建立日開始算 14 天
        due = dt + timedelta(days=14)
        return due.strftime("%b %d, %Y")
    except: return "ASAP"

# --- 2. Filter Targets ---
if 'df' not in globals() or df.empty:
    display(widgets.HTML("<h4 style='color:red'>⚠️ No data found. Run Cell 4 first.</h4>"))
else:
    pending_df = df[df['is_missing'] == True].copy()
    
    # ★★★ 重點修改：聚合時使用 'File_Created_Date' ★★★
    targets = pending_df.groupby(["Category", "App_Name", "reviewer", "folder_url"]).agg(
        missing_count=("is_missing", "count"),
        file_date=("File_Created_Date", "min") # 從 Last_Modified 改為 File_Created_Date
    ).reset_index()

    final_targets = []
    notes_db = manual_data_store if 'manual_data_store' in globals() else {}

    print("🔍 Looking up emails for pending reviewers...")
    
    for _, r in targets.iterrows():
        app_key = f"{r['Category']} > {r['App_Name']}"
        rev_key = r['reviewer']
        
        # Check Manual Overrides
        app_node = notes_db.get(app_key, {})
        if app_node.get("app_status") in ["Force Completed", "N/A"]: continue
        
        rev_override = app_node.get("reviewers", {}).get(rev_key, {}).get("override", "-")
        if rev_override in ["Mark Done", "Waived", "Escalated"]: continue
            
        final_targets.append({
            "App_Name": r['App_Name'], "reviewer": r['reviewer'],
            "missing": r['missing_count'], "folder_url": r['folder_url'],
            "due_date": calculate_due_date(r['file_date']),
            "email": get_email(r['reviewer'])
        })

    target_df = pd.DataFrame(final_targets)

    # --- 3. UI Construction (保持不變) ---
    clear_output()
    if target_df.empty:
        display(widgets.HTML("<h3>✅ Everyone is done! No emails needed.</h3>"))
    else:
        # (以下 UI 程式碼不變，與 V3 相同，略過以節省篇幅)
        # 這裡會直接使用上面計算好的 target_df
        default_subj = "[Action Required] Access Review: {app_name} (Due: {due_date})"
        w_subject_tpl = widgets.Text(value=default_subj, description="<b>Subject Tpl:</b>", layout=widgets.Layout(width='98%'))
        w_cc_global = widgets.Text(value="", description="<b>Global CC:</b>", placeholder="manager@company.com", layout=widgets.Layout(width='50%'))
        default_body = ("Hi {name},<br><br>This is a reminder that the access review for <b>{app_name}</b> is pending.<br>"
                        "Expected Completion: <b>{due_date}</b>.<br><br>"
                        "<ul><li>Pending items: {missing}</li>"
                        "<li>Link: <a href='{link}'>Click here to review</a></li></ul><br>"
                        "Please complete this as soon as possible.")
        w_body_tpl = widgets.Textarea(value=default_body.replace("<br>", "\n"), layout=widgets.Layout(width='100%', height='100px'))
        btn_update_preview = widgets.Button(description="🔄 Refresh Previews", button_style='info', layout=widgets.Layout(width='200px'))
        
        rows_container = []
        row_logic_map = {}
        
        display(widgets.HTML(f"<h3>📧 Email Sender ({len(target_df)} Users)</h3>"))
        
        for idx, r in target_df.iterrows():
            key = f"{idx}_{r['reviewer']}"
            ctx = {'name': r['reviewer'], 'missing': r['missing'], 'link': r['folder_url'], 'app_name': r['App_Name'], 'due_date': r['due_date']}
            
            w_chk = widgets.Checkbox(value=True, layout=widgets.Layout(width='30px'))
            email_color = "#0078d4" if r['email'] else "red"
            email_status = "" if r['email'] else "(Email not found)"
            info_html = (f"<div style='line-height:1.6; font-family:sans-serif;'>"
                         f"<span style='font-size:14px; color:#555'>📂 {r['App_Name']}</span><br>"
                         f"<span style='font-size:16px; font-weight:bold; color:{email_color}'>{r['reviewer']} {email_status}</span><br>"
                         f"<span style='font-size:13px; color:#d13438'>⚠️ Missing: <b>{r['missing']}</b> items</span><br>"
                         f"<span style='font-size:12px; color:#666'>📅 Due: {r['due_date']} (Created + 14d)</span></div>") # Updated Label
            
            w_info = widgets.HTML(info_html, layout=widgets.Layout(width='280px'))
            initial_body = default_body.format(**ctx)
            w_preview = widgets.HTML(value=f"<div style='font-family:sans-serif; font-size:12px; padding:8px'>{initial_body}</div>", layout=widgets.Layout(width='95%', height='120px', border='1px solid #e0e0e0', overflow_y='auto'))
            w_email = widgets.Text(value=r['email'], placeholder="Email", description="To:", layout=widgets.Layout(width='95%'))
            w_btn_send = widgets.Button(description="🚀 Send Email", button_style='warning', layout=widgets.Layout(width='95%'))
            
            row_logic_map[key] = {'ctx': ctx, 'w_preview': w_preview, 'w_email': w_email, 'w_btn': w_btn_send, 'w_chk': w_chk}
            
            def create_sender(k):
                def on_send(b):
                    data = row_logic_map[k]
                    if not data['w_email'].value: b.description = "❌ Missing Email"; return
                    b.disabled = True; b.description = "Sending..."
                    try:
                        final_body = w_body_tpl.value.replace("\n", "<br>").format(**data['ctx'])
                        final_subj = w_subject_tpl.value.format(**data['ctx']) if "{" in w_subject_tpl.value else w_subject_tpl.value
                        to_list = [{"emailAddress": {"address": data['w_email'].value}}]
                        cc_list = [{"emailAddress": {"address": w_cc_global.value.strip()}}] if w_cc_global.value.strip() else []
                        url = f"https://graph.microsoft.com/v1.0/users/{SENDER_EMAIL}/sendMail"
                        payload = {"message": {"subject": final_subj, "body": {"contentType": "HTML", "content": final_body}, "toRecipients": to_list, "ccRecipients": cc_list}}
                        resp = requests.post(url, headers={**headers, "Content-Type": "application/json"}, json=payload)
                        if resp.status_code == 202: b.button_style = 'success'; b.description = "✅ Sent"; data['w_chk'].value = False
                        else: b.button_style = 'danger'; b.description = "❌ Failed"; print(f"Err: {resp.text}")
                    except Exception as e: b.button_style = 'danger'; b.description = "❌ Error"; print(e)
                    finally: 
                        if b.description != "✅ Sent": b.disabled = False
                return on_send

            w_btn_send.on_click(create_sender(key))
            rows_container.append(widgets.HBox([widgets.HBox([w_chk, w_info], layout=widgets.Layout(width='30%')), widgets.VBox([widgets.Label("Body Preview:"), w_preview], layout=widgets.Layout(width='45%', padding='0 10px')), widgets.VBox([w_email, widgets.HTML("<br>"), w_btn_send], layout=widgets.Layout(width='25%', align_items='center'))], layout=widgets.Layout(height='150px', border='1px solid #ccc', margin='10px 0', padding='10px', background_color="#fcfcfc")))

        def update_previews(b):
            raw = w_body_tpl.value.replace("\n", "<br>")
            for k, d in row_logic_map.items():
                try: d['w_preview'].value = f"<div style='font-family:sans-serif; font-size:12px; padding:8px'>{raw.format(**d['ctx'])}</div>"
                except: pass
        btn_update_preview.on_click(update_previews)
        
        display(widgets.VBox([widgets.VBox([widgets.HTML("<b>Global Email Settings</b>"), w_subject_tpl, w_cc_global, widgets.Label("Body Template:"), widgets.HBox([w_body_tpl, btn_update_preview])], layout=widgets.Layout(padding='15px', border='2px solid #0078d4', margin='0 0 20px 0', background_color='#eef6fb')), widgets.VBox(rows_container)]))

In [ ]:
# Chat API test


import requests
import json

def retry_copilot_api():
    print("⚔️ 回合 2：權限補齊後，再次挑戰 Copilot Endpoint...")
    
    # 這是 Graph API 呼叫 Copilot 的標準進入點
    url = "https://graph.microsoft.com/beta/copilot/conversations"
    
    # 這裡嘗試帶入一個簡單的 prompt，有些 API 實作需要 body
    payload = {
        "promptText": "Hello Copilot, who am I?" 
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        
        print(f"📡 Status Code: {response.status_code}")
        
        if response.status_code in [200, 201]:
            print("🎉🎉🎉 成功了！我們連上了！")
            data = response.json()
            print(json.dumps(data, indent=2, ensure_ascii=False))
            return data
        else:
            # 如果還是失敗，看新的錯誤訊息
            print(f"❌ 依然失敗: {response.text}")
            
    except Exception as e:
        print(f"💥 發生錯誤: {e}")

# 執行
retry_copilot_api()

In [ ]:
# Chat box code 
# === Cell 4 (V12b): M365 Copilot 同步版 + 500 重試 & 詳細錯誤 ===
import ipywidgets as widgets
from IPython.display import display
import requests, json, time, re, uuid

GRAPH_ROOT = "https://graph.microsoft.com/beta"
# 你先前定義的 headers 至少要含 Authorization；這裡補 Content-Type
headers["Content-Type"] = "application/json"

debug_log = []
def log(msg):
    timestamp = time.strftime("%H:%M:%S")
    debug_log.append(f"[{timestamp}] {msg}")

def clean_bot_text(text):
    if not text: return ""
    text = text.replace("\n", "<br>")
    text = re.sub(r'\[(.*?)\]\((.*?)\)',
                  r'<a href="\\2" target="_blank" style="color:#0078d4">\\1</a>', text)
    return text

conv_id = None
def ensure_conversation():
    global conv_id
    if conv_id: return conv_id
    log("➕ 建立新對話")
    r = requests.post(f"{GRAPH_ROOT}/copilot/conversations", headers=headers, json={}, timeout=15)
    r.raise_for_status()
    conv_id = r.json()["id"]
    log(f"✅ conv_id: {conv_id}")
    return conv_id

def post_chat(cid, body):
    # 針對 500 做最多 3 次重試（指數退避）
    for attempt in range(3):
        try:
            r = requests.post(f"{GRAPH_ROOT}/copilot/conversations/{cid}/chat",
                              headers=headers, json=body, timeout=30)
            log(f"📥 Status: {r.status_code}")
            if r.status_code < 500:
                return r  # 成功或 4xx 直接回傳
        except Exception as e:
            log(f"⚠️ Exception: {e}")
        sleep = 2 ** attempt
        log(f"🔁 500/Exception，{sleep}s 後重試 ({attempt+1}/3)")
        time.sleep(sleep)
    return r  # 回傳最後一次結果

def chat_once(user_text, tz="America/Los_Angeles"):
    cid = ensure_conversation()
    body = {
        "message": { "text": user_text },
        "locationHint": { "timeZone": tz }  # 必填
        # 若要關掉 Web 搜尋可加：
        # "contextualResources": { "webContext": { "isWebEnabled": False } }
    }
    log("✉️ POST /chat")
    r = post_chat(cid, body)

    try:
        r.raise_for_status()
    except Exception:
        # 把更多細節塞回 UI
        err_json = ""
        try:
            err_json = json.dumps(r.json(), ensure_ascii=False)
        except:
            err_json = r.text
        req_id = r.headers.get("request-id", "n/a")
        cli_id = r.headers.get("client-request-id", "n/a")
        return f"❌ HTTP {r.status_code}<br>request-id: {req_id}<br>client-request-id: {cli_id}<br>body: {err_json}"

    msgs = r.json().get("messages", [])
    if not msgs: return "(empty response)"
    return msgs[-1].get("text", "(no text)")

# ========= UI =========
chat_history_list = []
ui_style = """<style> .chat-app { font-family:'Segoe UI',sans-serif;max-width:800px;margin:0 auto; }
.chat-window { background:#fff;border:1px solid #e0e0e0;border-radius:8px;height:400px;overflow-y:auto;padding:20px; }
.msg-row { margin-bottom:15px;display:flex;flex-direction:column; }
.msg-user { align-items:flex-end; } .msg-bot { align-items:flex-start; }
.bubble { padding:10px 16px;max-width:80%;line-height:1.5;font-size:15px;position:relative;word-wrap:break-word; }
.bubble-user { background:#0078D4;color:white;border-radius:18px 18px 4px 18px; }
.bubble-bot { background:#f3f2f1;color:#201f1e;border-radius:18px 18px 18px 4px;border:1px solid #e1dfdd; }
.debug-box { font-family:monospace;font-size:11px;color:#666;background:#f0f0f0;border:1px dashed #ccc;
             padding:8px;margin-top:10px;height:120px;overflow-y:auto; } code { background:#e1dfdd;padding:2px 6px;
             border-radius:3px;font-size:12px; } </style>"""

header_w = widgets.HTML("<h2>🤖 M365 Copilot (V12b - 同步 API)</h2>")
history_w = widgets.HTML(value=ui_style + "<div class='chat-app'><div class='chat-window'></div></div>")
debug_w = widgets.HTML(value="<div class='debug-box'>Ready.</div>")
input_w = widgets.Text(placeholder="試試輸入: Hello", layout=widgets.Layout(width='80%'))
send_btn_w = widgets.Button(description="Send", button_style='primary', layout=widgets.Layout(width='18%'))
loading_w = widgets.HTML(value="", layout=widgets.Layout(height='30px'))

def render_chat():
    history_w.value = f"{ui_style}<div class='chat-app'><div class='chat-window'>{''.join(chat_history_list)}</div></div>"

def update_debug():
    debug_w.value = "<div class='debug-box'>" + "<br>".join(debug_log[-12:]) + "</div>"

def on_chat_submit(b):
    user_text = input_w.value.strip()
    if not user_text: return
    debug_log.clear(); log("🏁 User send")
    chat_history_list.append(f"<div class='msg-row msg-user'><div class='bubble bubble-user'>{user_text}</div></div>")
    render_chat(); update_debug()
    input_w.value = ""; input_w.disabled = True; send_btn_w.disabled = True
    loading_w.value = "<span style='color:#0078D4; margin-left:10px;'>⏳ Copilot 正在回覆...</span>"
    try:
        reply = chat_once(user_text)
    except Exception as e:
        reply = f"❌ Error: {e}"
        log(str(e))
    update_debug()
    chat_history_list.append(f"<div class='msg-row msg-bot'><div class='bubble bubble-bot'>{clean_bot_text(reply)}</div></div>")
    render_chat()
    loading_w.value = ""; input_w.disabled = False; send_btn_w.disabled = False

send_btn_w.on_click(on_chat_submit)
input_w.on_submit(on_chat_submit)

display(widgets.VBox([header_w, history_w, loading_w, input_w, send_btn_w, debug_w]))